In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time

import gc
"""
Python中，主要依靠gc（garbage collector）模块的引用计数技术来进行垃圾回收。
所谓引用计数，就是考虑到Python中变量的本质不是内存中一块存储数据的区域，而是对一块内存数据区域的引用。
所以python可以给所有的对象（内存中的区域）维护一个引用计数的属性，
在一个引用被创建或复制的时候，让python把相关对象的引用计数+1；相反当引用被销毁的时候就把相关对象的引用计数-1。
当对象的引用计数减到0时，自然就可以认为整个python中不会再有变量引用这个对象，
所以就可以把这个对象所占据的内存空间释放出来了。
"""
from collections import Counter
import copy

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# 配置路径

In [2]:
data_path = 'data/'
tmp_res_path = 'tmp_results/'

# 读取数据集

In [3]:
# 读取数据函数
def read_data(file_name):

    df = pd.read_csv(data_path + file_name)
    df.drop_duplicates(inplace=True)
    
    return df

# 对数据进行内存压缩

In [4]:
# 节约内存的一个标配函数
def reduce_memory(df, verbose=True):
    
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'
          .format(end_mem, 100*(start_mem-end_mem)/start_mem, (time.time()-starttime)/60))
    return df

In [5]:
num_of_rows = None
"""
# train & test
test_data_path = data_path + 'test_format1.csv'
train_data_path = data_path + 'train_format1.csv'

# user_info & user_log
user_info_path = data_path + 'user_info_format1.csv'
user_log_path = data_path + 'user_log_format1.csv'
"""
# STEP1. read data from csv
# train & test
test_data = read_data(file_name='test_format1.csv')
train_data = read_data(file_name='train_format1.csv')

# user_info & user_log
user_info = read_data(file_name='user_info_format1.csv')
user_log = read_data(file_name='user_log_format1.csv')

# STEP2. reduce memory
train_data = reduce_memory(train_data)
test_data = reduce_memory(test_data)
user_info = reduce_memory(user_info)
user_log = reduce_memory(user_log)

-- Mem. usage decreased to  3.73 Mb (53.1% reduction),time spend:0.00 min
-- Mem. usage decreased to  5.49 Mb (31.2% reduction),time spend:0.00 min
-- Mem. usage decreased to  6.47 Mb (50.0% reduction),time spend:0.00 min
-- Mem. usage decreased to 981.69 Mb (60.9% reduction),time spend:0.13 min


# 数据处理

In [ ]:
# 合并训练&测试数据集
df_all_data = train_data.append(test_data)
df_all_data.head()

In [ ]:
# 合并用户信息&全量数据集
df_all_data = pd.merge(df_all_data, user_info, on='user_id', how='left')
df_all_data.head()

In [ ]:
# 清除垃圾
del train_data, test_data, user_info
gc.collect()

In [ ]:
# 按照时间排序
user_log = user_log.sort_values(by=['user_id', 'time_stamp'])
user_log.head()

In [ ]:
# 对于每个用户合并字段
# cat_id, seller_id, brand_id, time_stamp, action_type
list_join_func = lambda x: " ".join([str(i) for i in x])

agg_dict = {
    'item_id': list_join_func,
    'cat_id': list_join_func,
    'seller_id': list_join_func,
    'brand_id': list_join_func, 
    'time_stamp': list_join_func, 
    'action_type': list_join_func
}

rename_dict = {
    'item_id': 'item_path',
    'cat_id': 'cat_path',
    'seller_id': 'seller_path',
    'brand_id': 'brand_path', 
    'time_stamp': 'time_stamp_path', 
    'action_type': 'action_type_path'
}

def merga_list(df_id, join_col, df_data, agg_dict, rename_dict):
    
    df_data = df_data.groupby(join_col).agg(agg_dict).reset_index().rename(columns=rename_dict)
    df_id = pd.merge(df_id, df_data, on=join_col, how='left')
    
    return df_id
    

df_all_data = merga_list(df_all_data, 'user_id', user_log, agg_dict, rename_dict)
df_all_data.head()

In [ ]:
# 删除数据并回收内存
del user_log
gc.collect()

# 定义特征统计函数
## 定义统计函数

In [ ]:
# total count of data
def cnt_(x):
    try:
        return len(x.split(' '))
    except:
        return -1

# number of unique element of data
def nunique_(x):
    try:
        return len(set(x.split(' ')))
    except:
        return -1
    
# maximum of data
def max_(x):
    try:
        return np.max([float(i) for i in x.split(' ')])
    except:
        -1
        
# minimum of data
def min_(x):
    try:
        return np.min([float(i) for i in x.split(' ')])
    except:
        -1
        
# standard deviation of data
def std_(x):
    try:
        return np.std([float(i) for i in x.split(' ')])
    except:
        -1
        
# the top k element of data
def most_n(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][0]
    except:
        -1
        
# total count of the top k element of data
def most_n_cnt(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][1]
    except:
        -1


In [ ]:
Counter('1 2 8 10 13 20 20'.split(' ')).most_common(3)[3-1][0]   

## 调用统计函数

In [ ]:
def user_cnt(df_data, single_col, name):
    """

    @param df_data: DataFrame
    @param single_col: the col name to apply stats
    @param name: new col name to create
    @return:
    @rtype:
    """
    df_data[name] = df_data[single_col].apply(cnt_)
    
    return df_data


def user_nunique(df_data, single_col, name):
    """

    @param df_data: DataFrame
    @param single_col: the col name to apply stats
    @param name: new col name to create
    @return:
    @rtype:
    """
    df_data[name] = df_data[single_col].apply(nunique_)
    
    return df_data


def user_max(df_data, single_col, name):
    """

    @param df_data: DataFrame
    @param single_col: the col name to apply stats
    @param name: new col name to create
    @return:
    @rtype:
    """
    df_data[name] = df_data[single_col].apply(max_)
    
    return df_data


def user_min(df_data, single_col, name):
    """

    @param df_data: DataFrame
    @param single_col: the col name to apply stats
    @param name: new col name to create
    @return:
    @rtype:
    """
    df_data[name] = df_data[single_col].apply(min_)
    
    return df_data


def user_std(df_data, single_col, name):
    """

    @param df_data: DataFrame
    @param single_col: the col name to apply stats
    @param name: new col name to create
    @return:
    @rtype:
    """
    df_data[name] = df_data[single_col].apply(std_)
    
    return df_data


def user_most_n(df_data, single_col, name, n=1):
    """

    @param df_data: DataFrame
    @param single_col: the col name to apply stats
    @param name: new col name to create
    @return:
    @rtype:
    """
    func = lambda x: most_n(x, n)
    df_data[name] = df_data[single_col].apply(func)
    
    return df_data


def user_most_n_cnt(df_data, single_col, name, n=1):
    """

    @param df_data: DataFrame
    @param single_col: the col name to apply stats
    @param name: new col name to create
    @return:
    @rtype:
    """
    func = lambda x: most_n_cnt(x, n)
    df_data[name] = df_data[single_col].apply(func)
    
    return df_data


## 提取统计特征
### 特征统计
用户复购行为，考虑用户自身因素和商家因素，一般来说是商家提供的产品和服务能较好地满足用户需求；
#### 店铺特征
统计基于商户的特征主要目的是分析商户在当前市场的受欢迎程度及商户自身对忠实用户的吸引力


In [ ]:
"""
店铺的基本统计特征: 店铺/商品/品牌
"""
# 选取数据测试/全量数据开关
df_all_data_test = df_all_data.head(1)  # 少量数据测试代码逻辑
# df_all_data_test = df_all_data  # 全量数据构建特征
print('len', len(df_all_data_test))
print(df_all_data_test.columns)

In [ ]:
"""
用户点击、浏览、加入购物车、购买行为统计
"""
# 1. 每个用户交互过的商店个数
df_all_data_test = user_cnt(df_all_data_test, 'seller_path', 'user_cnt')
# 2. 每个用户交互过的不同商店个数
df_all_data_test = user_nunique(df_all_data_test, 'seller_path', 'seller_nunique')
# 3. 每个用户交互过的不同商品品类个数
df_all_data_test = user_nunique(df_all_data_test, 'cat_path', 'cat_nunique')
# 4. 每个用户交互过的不同品牌个数
df_all_data_test = user_nunique(df_all_data_test, 'brand_path', 'brand_nunique')
# 5. 每个用户交互过的不同商品个数
df_all_data_test = user_nunique(df_all_data_test, 'item_path', 'item_nunique')
# 6. 每个用户活跃的天数
df_all_data_test = user_nunique(df_all_data_test, 'time_stamp_path', 'time_stamp_nunique')
# 7. 每个用户不同行为种类个数
df_all_data_test = user_nunique(df_all_data_test, 'action_type_path', 'action_type_nunique')

In [ ]:
df_all_data_test.head()

In [ ]:
# time_tamp:购买时间（格式：mmdd）
# 8. 每个用户最近一次活跃时间 
df_all_data_test = user_max(df_all_data_test, 'time_stamp_path', 'time_stamp_max')
# 9. 每个用户第一次活跃时间
df_all_data_test = user_min(df_all_data_test, 'time_stamp_path', 'time_stamp_min')
# 10. 活跃时间方差
df_all_data_test = user_std(df_all_data_test, 'time_stamp_path', 'time_stamp_std')
# 11. 最早和最晚日期相隔天数
df_all_data_test['time_stamp_range'] = df_all_data_test['time_stamp_max'] - df_all_data_test['time_stamp_min']


In [ ]:
# 12. 用户最喜欢的商店(交互过最多的seller_id)
df_all_data_test = user_most_n(df_all_data_test, 'seller_path', 'seller_most_1', n=1)
# 13. 用户最喜欢的品类(交互过最多的cat_id)
df_all_data_test = user_most_n(df_all_data_test, 'cat_path', 'cat_most_1', n=1)
# 14. 用户最喜欢的品牌(交互过最多的brand_id)
df_all_data_test = user_most_n(df_all_data_test, 'brand_path', 'brand_most_1', n=1)
# 15. 用户最喜欢的做的行为动作(最多的action_type)
df_all_data_test = user_most_n(df_all_data_test, 'action_type_path', 'action_type_1', n=1)

# 16. 用户最喜欢的商店交互次数(交互过最多的seller_id)
df_all_data_test = user_most_n_cnt(df_all_data_test, 'seller_path', 'seller_most_1_cnt', n=1)
# 17. 用户最喜欢的品类交互次数(交互过最多的cat_id)
df_all_data_test = user_most_n_cnt(df_all_data_test, 'cat_path', 'cat_most_1_cnt', n=1)
# 18. 用户最喜欢的品牌交互次数(交互过最多的brand_id)
df_all_data_test = user_most_n_cnt(df_all_data_test, 'brand_path', 'brand_most_1_cnt', n=1)
# 19. 用户最喜欢的做的行为动作交互次数(最多的action_type)
df_all_data_test = user_most_n_cnt(df_all_data_test, 'action_type_path', 'action_type_1_cnt', n=1)


#### 用户特征
- 统计用户交互次数
- 用户交互次数在所有用户中的对比
- 用户点击次数在所有用户中的对比
- 用户加入购物车次数在所有用户加入购物车次数中的地位
- 用户购买次数在所有用户购买次数中的地位
- 用户收藏次数在所有用户收藏次数中的地位
- 统计用户不同行为的习惯
- 统计用户的点击、加入购物车、收藏的购买转化率
- 用户交互的时间信息（按天、次数）
- 用户的活跃程度的变化

In [ ]:
"""
用户统计特征: 点击、加入购物车、购买、收藏夹
"""

"""
copy & deepcopy
我们寻常意义的复制就是深复制，即将被复制对象完全再复制一遍作为独立的新个体单独存在。
所以改变原有被复制对象不会对已经复制出来的新对象产生影响。
—–而浅复制并不会产生一个独立的对象单独存在，他只是将原有的数据块打上一个新标签，
所以当其中一个标签被改变的时候，数据块就会发生变化，另一个标签也会随之改变。
"""
# 各种行为的店铺数量统计
def col_cnt_(df_data, columns_list, action_type):
    """
    统计函数定义
    根据不同行为的业务函数提取不同特征, 对不同的行为进行分别统计
    action_type包含{0, 1, 2, 3}，0表示单击，1表示添加到购物车，2表示购买，3表示添加到收藏夹
    
    @param df_data: DataFrame
    @param columns_list: list, column name to stats
    @param action_type: 
    @return:
    @rtype:
    """
    try:
        data_dict = {}

        col_list = copy.deepcopy(columns_list)  # deepcopy
        if action_type != None:
            # 加上需要统计的目标col: action_type_path
            col_list += ['action_type_path']

        for col in col_list:
            # 将每一个用户col内的元素以list的形式存入字典 
            # 结果类似于{'seller_path': ['3152', '3152', '3022', ...], 'action_type_path': ['3', '3', '0', ...]}
            data_dict[col] = df_data[col].split(' ')

        path_len = len(data_dict[col])  # 统计每一行用户的行为个数总和
        data_out = []
        for i_ in range(path_len):
            data_txt = ''
            for col_ in columns_list:
                if data_dict['action_type_path'][i_] == action_type:
                    # 如果是指定的action_type, 就把对应的col内容添加到data_txt中
                    data_txt += '_' + data_dict[col_][i_]
                # 如果不是的话, 就跳过, data_txt为空字符串
            data_out.append(data_txt)
        return len(data_out)
    
    except:
        return -1
    
    
def user_col_cnt(df_data, columns_list, action_type, name):
    """
    统计函数调用
    各种行为的店铺数量统计
    action_type包含{0, 1, 2, 3}，0表示单击，1表示添加到购物车，2表示购买，3表示添加到收藏夹
    
    @param df_data: DataFrame
    @param columns_list: 
    @param action_type: 
    @return:
    @rtype:
    """
    df_data[name] = df_data.apply(lambda x: col_cnt_(x, columns_list, action_type), axis=1)
    
    return df_data


# 各种行为的不同店铺数量统计
def col_nuique_(df_data, columns_list, action_type):
    try:
        data_dict = {}

        col_list = copy.deepcopy(columns_list)
        if action_type != None:
            # 加上需要统计的目标col: action_type_path
            col_list += ['action_type_path']
        for col in col_list:
            # 将每一个用户col内的元素以list的形式存入字典 
            data_dict[col] = df_data[col].split(' ')

        path_len = len(data_dict[col])
        data_out = []
        for i_ in range(path_len):
            data_txt = ''
            for col_ in columns_list:
                if data_dict['action_type_path'][i_] == action_type:
                    data_txt += '_' + data_dict[col_][i_]
            data_out.append(data_txt)
        
        # set用于去重
        return len(set(data_out))
    except:
        return -1
    

def user_col_nunique(df_data, columns_list, action_type, name):
    """
    统计函数调用
    各种行为的不同店铺数量统计
    action_type包含{0, 1, 2, 3}，0表示单击，1表示添加到购物车，2表示购买，3表示添加到收藏夹
    
    @param df_data: DataFrame
    @param columns_list: 
    @param action_type: 
    @return:
    @rtype:
    """
    df_data[name] = df_data.apply(lambda x: col_nuique_(x, columns_list, action_type), axis=1)

    return df_data
    

In [ ]:
"""
统计店铺被用户点击次数，加购次数，购买次数，收藏次数
"""

# (0: 表示单击) 用户对商店商品的点击次数
df_all_data_test = user_col_cnt(df_all_data_test,  ['seller_path'], '0', 'user_cnt_0')
# (1: 表示添加到购物车) 用户对商店商品的添加购物车次数
df_all_data_test = user_col_cnt(df_all_data_test,  ['seller_path'], '1', 'user_cnt_1')
# (2: 表示购买) 用户对商店的购买商品次数
df_all_data_test = user_col_cnt(df_all_data_test,  ['seller_path'], '2', 'user_cnt_2')
# (3: 表示添加到收藏夹) 用户对商店商品的添加购物车次数
df_all_data_test = user_col_cnt(df_all_data_test,  ['seller_path'], '3', 'user_cnt_3')



#### 组合特征

In [ ]:
# 点击商店中物品的次数
df_all_data_test = user_col_cnt(df_all_data_test,  ['seller_path', 'item_path'], '0', 'user_cnt_0')

# 点击不同商店中不同物品的次数
df_all_data_test = user_col_nunique(df_all_data_test,  ['seller_path', 'item_path'], '0', 'seller_nunique_0')



**目前提取的特征**
- label : 是否复购的标示
- merchant_id : 商店id
- prob : 给定客户是给定商家的重复购买者的概率，取值在[0, 1]
- user_id : 用户id
- age_range : 用户年龄范围(分类)
- gender : 用户性别
- item_path : 用户交互过的商品id集合
- cat_path : 用户交互过的商品品类id集合
- seller_path : 用户交互过的商店id集合
- brand_path : 用户交互过的品牌id集合
- time_stamp_path : 用户购买时间集合 (格式：mmdd)
- action_type_path : 用户交互行为类型集合
- user_cnt : 用户交互过的商店个数
- seller_nunique : 用户交互过的不同商店个数
- cat_nunique : 用户交互过的不同品类个数
- brand_nunique : 用户交互过的不同品牌个数
- item_nunique : 用户交互过的不同商品个数
- time_stamp_nunique : 用户活跃的天数
- action_type_nunique : 用户不同行为种类个数
- time_stamp_max : 用户最近一次活跃ts (格式: MMDD)
- time_stamp_min : 用户最早一次活跃ts (格式: MMDD)
- time_stamp_std : 用户最早一次活跃ts标准差
- time_stamp_range : 用户最早一次活跃与最近一次活跃日期差值 (非天数, 数值本身没有意义, 但是不同用户间可以比大小)
- seller_most_1 : 用户交互最多次的商店
- cat_most_1 : 用户交互最多次的商品品类
- brand_most_1 : 用户交互最多次的品牌
- action_type_1 : 用户最多次的交互行为
- seller_most_1_cnt : 用户交互最多次的商店 交互次数
- cat_most_1_cnt : 用户交互最多次的商品品类 交互次数
- brand_most_1_cnt : 用户交互最多次的品牌 交互次数
- action_type_1_cnt : 用户最多次的交互行为 次数
- user_cnt_0 : 用户点击商店中物品的次数
- user_cnt_1 : 用户对商店商品的添加购物车次数
- user_cnt_2 : 用户对商店商品的购买次数
- user_cnt_3 : 用户对商店商品的添加购物车次数
- seller_nunique_0 : 点击不同商店中不同物品的个数



# 利用countvector, tfidf提取特征

## 词袋模型TF-IDF
[参考资料: sklearn-TfidfVectorizer彻底说清楚](https://zhuanlan.zhihu.com/p/67883024)
<br>
<br>
- **TF （Term Frequency）—— “单词频率”**
<br>
1. 意思就是说，我们计算一个查询关键字中某一个单词在目标文档中出现的次数。举例说来，如果我们要查询 “Car Insurance”，那么对于每一个文档，我们都计算“Car” 这个单词在其中出现了多少次，“Insurance”这个单词在其中出现了多少次。这个就是 TF 的计算方法。
<br>
2. TF背后的隐含的假设是，查询关键字中的单词应该相对于其他单词更加重要，而文档的重要程度，也就是相关度，与单词在文档中出现的次数成正比。比如，“Car” 这个单词在文档 A 里出现了 5 次，而在文档 B 里出现了 20 次，那么 TF 计算就认为文档 B 可能更相关。
<br>
3. 然而，仅有 TF 不能比较完整地描述文档的相关度。因为语言的因素，有一些单词可能会比较自然地在很多文档中反复出现，比如英语中的 “The”、“An”、“But” 等等。这些词大多起到了链接语句的作用，是保持语言连贯不可或缺的部分。然而，如果我们要搜索 “How to Build A Car” 这个关键词，其中的 “How”、“To” 以及 “A” 都极可能在绝大多数的文档中出现，这个时候 TF 就无法帮助我们区分文档的相关度了。

- **IDF（Inverse Document Frequency）—— “逆文档频率”**
<br>
1. 思路就是我们需要去 “惩罚”（Penalize）那些出现在太多文档中的单词。也就是说，真正携带 “相关” 信息的单词仅仅出现在相对比较少，有时候可能是极少数的文档里。
<br>
2. 这个信息，很容易用 “文档频率” 来计算，也就是，有多少文档涵盖了这个单词。很明显，如果有太多文档都涵盖了某个单词，这个单词也就越不重要，或者说是这个单词就越没有信息量。
<br>
3. 因此，我们需要对 TF 的值进行修正，而 IDF 的想法是用 DF 的倒数来进行修正。倒数的应用正好表达了这样的思想，DF 值越大越不重要。


In [ ]:
"""
文本表示模型: 利用countvector，tfidf提取特征
"""
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from scipy import sparse

[Python sklearn 中的TfidfVectorizer参数解析](https://blog.csdn.net/weixin_42462804/article/details/105433680)
- stop_words: string, list, or None(default), 如果未english，用于英语内建的停用词列表，如果未list，该列表被假定为包含停用词，列表中的所有词都将从令牌中删除， 如果None，不使用停用词。
- max_df: 可以被设置为范围[0.7, 1.0)的值，基于内部预料词频来自动检测和过滤停用词。
- ngram_range (min,max): 是指将text分成min，min+1，min+2,.........max 个不同的词组。比如'Python is useful'中ngram_range(1,3)之后可得到'Python' 'is' 'useful' 'Python is' 'is useful' 和'Python is useful'如果是ngram_range(1,1)则只能得到单个单词'Python' 'is'和'useful'
- analyzer: string，{'word', 'char'} or callable定义特征为词（word）或n-gram字符
- max_df: float in range [0.0, 1.0] or int, optional, 1.0 by default当构建词汇表时，严格忽略高于给出阈值的文档频率的词条，语料指定的停用词。如果是浮点值，该参数代表文档的比例，整型绝对计数值，如果词汇表不为None，此参数被忽略。
- binary: boolean， False by default, 如果为True，所有非零计数被设置为1，这对于离散概率模型是有用的，建立二元事件模型，而不是整型计数。
- token_pattern: 正则表达式显示了”token“的构成，仅当analyzer == ‘word’时才被使用。
- sublinear_tf: boolean， optional应用线性缩放TF，例如，使用1+log(tf)覆盖tf



In [ ]:
"""
TF-IDF
"""
# 实例化tf实例
tfidfVec = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS,  # 英语停用词
                           ngram_range=(1, 1),  # 指将text分成min，min+1，min+2,.........max 个不同的词组
                           max_features=100)

# 对用户交互过商店序列进行特征提取 ==> 用户交互过的商店序列向量化
columns_list = ['seller_path']
for i, col in enumerate(columns_list):
    df_all_data_test[col] = df_all_data_test[col].astype(str)
    tfidfVec.fit(df_all_data_test[col])
    # 训练, 构建词汇表以及词项idf值, 并将输入文本列表转成VSM矩阵形式
    data_ = tfidfVec.transform(df_all_data_test[col])
    if i == 0:
        data_cat = data_
    else:
        # 数据拼接
        data_cat = sparse.hstack((data_cat, data_))  # Stack sparse matrices horizontally (column wise)



In [ ]:
"""
特征重命名以及特征合并
"""
df_tfidf = pd.DataFrame(data_cat.toarray())
df_tfidf.columns = ['tfidf_' + str(i) for i in df_tfidf.columns]
df_all_data_test = pd.concat([df_all_data_test, df_tfidf], axis=1)


# 嵌入特征 embedding

In [ ]:
import gensim

[gensim.models.word2vec参数说明](https://blog.csdn.net/u011748542/article/details/85880852)
- sentences (iterable of iterables, optional) – 供训练的句子，可以使用简单的列表，但是对于大语料库，建议直接从磁盘/网络流迭代传输句子。参阅word2vec模块中的BrownCorpus，Text8Corpus或LineSentence。
- corpus_file (str, optional) – LineSentence格式的语料库文件路径。
- size (int, optional) – word向量的维度。
- window (int, optional) – 一个句子中当前单词和被预测单词的最大距离。
- min_count (int, optional) – 忽略词频小于此值的单词。
- workers (int, optional) – 训练模型时使用的线程数。
- sg ({0, 1}, optional) – 模型的训练算法: 1: skip-gram; 0: CBOW.
- hs ({0, 1}, optional) – 1: 采用hierarchical softmax训练模型; 0: 使用负采样。
- negative (int, optional) – > 0: 使用负采样，设置多个负采样(通常在5-20之间)。
- ns_exponent (float, optional) – 负采样分布指数。1.0样本值与频率成正比，0.0样本所有单词均等，负值更多地采样低频词。
- cbow_mean ({0, 1}, optional) – 0: 使用上下文单词向量的总和; 1: 使用均值，适用于使用CBOW。
- alpha (float, optional) – 初始学习率。
- min_alpha (float, optional) – 随着训练的进行，学习率线性下降到min_alpha。
- seed (int, optional) – 随机数发生器种子。
- max_vocab_size (int, optional) – 词汇构建期间RAM的限制; 如果有更多的独特单词，则修剪不常见的单词。 每1000万个类型的字需要大约1GB的RAM。
- max_final_vocab (int, optional) – 自动选择匹配的min_count将词汇限制为目标词汇大小。
- sample (float, optional) – 高频词随机下采样的配置阈值，范围是(0,1e-5)。
- hashfxn (function, optional) – 哈希函数用于随机初始化权重，以提高训练的可重复性。
- iter (int, optional) – 迭代次数。
- trim_rule (function, optional) – 词汇修剪规则，指定某些词语是否应保留在词汇表中，修剪掉或使用默认值处理。
- sorted_vocab ({0, 1}, optional) – 如果为1，则在分配单词索引前按降序对词汇表进行排序。
- batch_words (int, optional) – 每一个batch传递给线程单词的数量。
- compute_loss (bool, optional) – 如果为True，则计算并存储可使用get_latest_training_loss()检索的损失值。
- callbacks (iterable of CallbackAny2Vec, optional) – 在训练中特定阶段执行回调序列。

In [ ]:
# train w2v model
"""
size=100: word向量的维度为100
window=5: 一个句子中当前单词和被预测单词的最大距离为5
min_count=5: 忽略词频小于5的单词
"""
model = gensim.models.Word2Vec(df_all_data_test['seller_path'].apply(lambda x: x.split(' ')),
                               size=100, window=5, min_count=5, workers=4)
# save model to local
# model.save(tmp_res_path + 'product2vec.model')
# model = gensim.models.Word2Vec.load(tmp_res_path + "seller2vec.model")

In [ ]:
def mean_w2v_(x, model, size=100):
    """
    
    @param x: 
    @type x:
    @param model:
    @type model:
    @param size:
    @type size: int
    @return:
    @rtype:
    """
    try:
        i = 0
        for word in x.split(' '):
            # word: each seller in col 'seller_path'
            if word in model.wv.vocab:
                # if the seller in model
                i += 1
                if i == 1:
                    # if the first seller, initial
                    vec = np.zeros(size)
                # sum the vector
                vec += model.wv[word]
        # calculate the sum of vectors
        return vec / i
    
    except:
        return np.zeros(size)

    
def get_mean_w2v(df_data, columns, model, size):
    """
    
    @param df_data: 
    @type df_data: DataFrame
    @param columns: column name
    @type columns: string
    @param model:
    @type model: 
    @param size: size of embedding vector
    @type size: int
    @return: 
    @rtype: DataFrame
    """
    data_array = []
    
    for index, row in df_data.iterrows():
        # for each row idx and content
        w2v = mean_w2v_(row[columns], model, size)
        data_array.append(w2v)
    
    return pd.DataFrame(data_array)


In [ ]:
df_embedding = get_mean_w2v(df_all_data_test, 'seller_path', model, 100)
# rename emdding column name
df_embedding.columns = ['embedding_' + str(i) for i in df_embedding.columns]

df_all_data_test = pd.concat([df_all_data_test, df_embedding], axis=1)


# Stacking分类特征

## stacking package

In [ ]:
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from scipy import sparse
import xgboost
import lightgbm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss, mean_absolute_error, mean_squared_error
from sklearn.naive_bayes import MultinomialNB, GaussianNB